In [11]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai chromadb 
!pip install --upgrade --quiet pypdf pandas streamlit python-dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from dotenv import load_dotenv

In [1]:
import os
import tempfile
import streamlit as st
import pandas as pd

In [7]:
load_dotenv()

True

## DEFINE OUR LLM

In [9]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm.invoke("Tell me a joke about cats")


AIMessage(content='Why did the cat sit on the computer?\n\nBecause it wanted to keep an eye on the mouse!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CRw4pFNz02Dk5RHGNtuvqINoj4FIP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5d8a569b-d0b5-473c-a244-a0c1e859fbfe-0', usage_metadata={'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Process PDF document

In [10]:
loader = PyPDFLoader("data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf")

In [11]:
pages = loader.load()
pages

Ignoring wrong pointing object 23 0 (offset 0)


[Document(metadata={'producer': 'Mac OS X 10.13.5 Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20180624213431Z00'00'", 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'moddate': "D:20180624213431Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='APPLIED COGNITIVE PSYCHOLOGY\nAppl. Cognit. Psychol. 20: 139–156 (2006)\nPublished online 31 October 2005 in Wiley InterScience\n(www.interscience.wiley.com) DOI: 10.1002/acp.1178\nConsequences of Erudite Vernacular Utilized Irrespective\nof Necessity: Problems with Using Long Words Needlessly\nDANIEL M. OPPENHEIMER*\nPrinceton University, USA\nSUMMARY\nMost texts on writing style encourage authors to avoid overly-complex words. However, a majority\nof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to give\nthe impression of intelligence. This pa

## Split Document

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, 
                                               chunk_overlap=200, 
                                               length_function=len, 
                                               separators=["\n\n", "\n", " "])

In [16]:
chunks = text_splitter.split_documents(pages)
chunks

[Document(metadata={'producer': 'Mac OS X 10.13.5 Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20180624213431Z00'00'", 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'moddate': "D:20180624213431Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='APPLIED COGNITIVE PSYCHOLOGY\nAppl. Cognit. Psychol. 20: 139–156 (2006)\nPublished online 31 October 2005 in Wiley InterScience\n(www.interscience.wiley.com) DOI: 10.1002/acp.1178\nConsequences of Erudite Vernacular Utilized Irrespective\nof Necessity: Problems with Using Long Words Needlessly\nDANIEL M. OPPENHEIMER*\nPrinceton University, USA\nSUMMARY\nMost texts on writing style encourage authors to avoid overly-complex words. However, a majority\nof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to give\nthe impression of intelligence. This pa

## Create Embeddings

In [17]:
def get_embedding():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002"
    )
    return embeddings

embedding_function = get_embedding()

In [18]:
test_vector = embedding_function.embed_query("cat")
test_vector

[-0.0070539116859436035,
 -0.01734057068824768,
 -0.009698242880403996,
 -0.03073945827782154,
 -0.012484360486268997,
 0.0030714645981788635,
 -0.005111427512019873,
 -0.041118279099464417,
 -0.014561542309820652,
 -0.021268075332045555,
 0.019240519031882286,
 0.05075980722904205,
 -0.0012246867408975959,
 0.00255216914229095,
 -0.03845268115401268,
 -0.006057857070118189,
 0.035475149750709534,
 -0.004622261971235275,
 0.002374935196712613,
 -0.013455602340400219,
 -0.01894276589155197,
 0.00905311107635498,
 0.015894342213869095,
 -0.00870573241263628,
 -0.014731687493622303,
 0.0071425288915634155,
 0.013150759972631931,
 -0.013228743337094784,
 0.0028676455840468407,
 0.0048987469635903835,
 0.004033844918012619,
 -0.016801780089735985,
 -0.015752553939819336,
 -0.04304658621549606,
 -0.027123885229229927,
 -0.004278427921235561,
 0.008074779063463211,
 -0.009939280338585377,
 0.022076262161135674,
 -0.009124004282057285,
 0.004920014645904303,
 0.00036133575486019254,
 -0.012073

In [19]:
# to calculate distance b/w two pieces of texts(amsterdem and coffeshop)
from langchain.evaluation import load_evaluator
evaluator = load_evaluator(evaluator="embedding_distance", embeddings=embedding_function)
evaluator.evaluate_strings(prediction="Amsterdem", reference="coffeshop")


{'score': 0.20157103463556092}

## Create Vector Database

In [20]:
vectorstore = Chroma.from_documents(documents=chunks, 
                                    embedding=embedding_function, persist_directory="vectorstore")
vectorstore.persist()

C:\Users\Codes Soft\AppData\Local\Temp\ipykernel_17676\165137520.py:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [21]:

import uuid
def create_vectorstore(chunks, embedding_function, vectorstore_path):
    #create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    #ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    for chunk, id in zip(chunks, ids):
        if id not in unique_ids:
            unique_ids.add(id)
            unique_chunks.append(chunk)

    #create a new chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids = list(unique_ids),
                                        embedding=embedding_function, persist_directory=vectorstore_path)
    vectorstore.persist()
    return vectorstore

In [22]:
#create vectorstore 
vectorstore = create_vectorstore(chunks=chunks,
                                 embedding_function=embedding_function,
                                 vectorstore_path="vectorstore_chroma")


## Query for relevant data

In [23]:
#load vectorstore
vectorstore = Chroma(persist_directory= "vectorstore_chroma",                 embedding_function=embedding_function)

C:\Users\Codes Soft\AppData\Local\Temp\ipykernel_17676\659172616.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory= "vectorstore_chroma",                 embedding_function=embedding_function)


In [24]:
#create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity")
#uses cosine distance to determine similarity   
relevant_chunks = retriever.invoke("What is the title of the article?")
relevant_chunks


[Document(metadata={'total_pages': 18, 'page_label': '15', 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'producer': 'Mac OS X 10.13.5 Quartz PDFContext', 'creator': 'Preview', 'page': 14, 'keywords': '', 'creationdate': "D:20180624213431Z00'00'", 'aapl:keywords': '[]', 'moddate': "D:20180624213431Z00'00'"}, page_content='anonymous reviewers for advice and support.\nREFERENCES\nBem, D. J. (1995). Writing a review article for Psychological Bulletin. Psychological Bulletin, 118 ,\n172–177.\nChang, P. M. Y. (1993). An institutional analysis of the evolution of the denominational system in\nAmerican Protestantism, 1790–1980. Unpublished doctoral dissertation, Stanford University.\nDescartes, R. (1990). Meditations on ﬁrst philosophy (G. Heffernan, Trans.). London: University of\nNotre Dame Press. (Original work published 1641).\nDescartes, R. (1993). Meditations on First Philosophy (S. Tweyman, Trans.). Lon

In [25]:
#prompt_template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to anwer the question. If you dont know the answer, say that  you dont know. DONT MAKE UP ANYTHING.

{context}

----
Answer the question based on the above context: {question}
"""


In [26]:
#Concatenate context text
context_text = "\n\n----\n\n".join([doc.page_content for doc in relevant_chunks])

#create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context = context_text, question="What is the title of the paper?")
print(prompt)


Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to anwer the question. If you dont know the answer, say that  you dont know. DONT MAKE UP ANYTHING.

anonymous reviewers for advice and support.
REFERENCES
Bem, D. J. (1995). Writing a review article for Psychological Bulletin. Psychological Bulletin, 118 ,
172–177.
Chang, P. M. Y. (1993). An institutional analysis of the evolution of the denominational system in
American Protestantism, 1790–1980. Unpublished doctoral dissertation, Stanford University.
Descartes, R. (1990). Meditations on ﬁrst philosophy (G. Heffernan, Trans.). London: University of
Notre Dame Press. (Original work published 1641).
Descartes, R. (1993). Meditations on First Philosophy (S. Tweyman, Trans.). London: Routledge.
(Original work published 1641).
Einhorn, H., & Hogarth, R. (1986). Judging probable cause. Psychological Bulletin, 99 , 3–19.
Festinger, L. (1957). A theory of cognitive dissonance . Stanford, C

In [27]:
llm.invoke(prompt)

AIMessage(content="The title of the paper is not provided in the retrieved context. Therefore, I don't know the answer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 1195, 'total_tokens': 1216, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CRw6GKrxs5drrcdQtNprvMcUoH2Rx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--fbcf51e5-c896-46a0-887d-f4adfd159d6e-0', usage_metadata={'input_tokens': 1195, 'output_tokens': 21, 'total_tokens': 1216, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Using Langchain Expression Language

In [28]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm   
        )

## Generate Structured Responses

In [37]:
#using pydantic class(a data vaidation library)
# class ExtractedInfo(BaseModel):
#     """Extracted Information about the research article"""
#     paper_title: str = Field(description="Title of the paper")
#     paper_summary: str = Field(description="Summary of the paper")
#     publication_year: int = Field(description="Year of publication of the paper")
#     paper_author: str = Field(description="Nmaes of the authors of the paper")




class AnswerWithSources(BaseModel):
    """An answer to the question , with sources and reasoning"""
    answer: str = Field(description="Answer to question")
    sources: str = Field("Full direct text chunk from the context used to answer the question")
    reasoning: str = Field("Explain the reasoning of the answer based on the sources")




class ExtractedInfo(BaseModel):
    """Extracted information about the research article"""
    paper_title: AnswerWithSources
    paper_summary:  AnswerWithSources
    publication_year:  AnswerWithSources
    paper_authors:  AnswerWithSources


In [38]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm.with_structured_output(ExtractedInfo, strict=True)

        )

rag_chain.invoke("Give me the title , summary, publication date, authors of the research paper.")

ExtractedInfo(paper_title=AnswerWithSources(answer='Problems with long words', sources='Oppenheimer, D. M. (2006). Problems with long words. Applied Cognitive Psychology, 20, 139–156.', reasoning="The document included an excerpt and reference to the paper 'Problems with long words' by Oppenheimer."), paper_summary=AnswerWithSources(answer='This study investigates the effects of using complex language in writing on the perceived intelligence of the author and the difficulty of understanding the text.', sources='Oppenheimer, D. M. (2006). Problems with long words. Applied Cognitive Psychology, 20, 139–156.', reasoning="The context discusses how participants rated the intelligence of authors and difficulty of their passages, indicating the study's focus on language complexity."), publication_year=AnswerWithSources(answer='2006', sources='Oppenheimer, D. M. (2006). Problems with long words. Applied Cognitive Psychology, 20, 139–156.', reasoning='The reference clearly states the publicatio

## Transform Response into a dataframe

In [40]:
structured_response = rag_chain.invoke("Give me the title,summary, publication date, authors of the research paper.")

df = pd.DataFrame([structured_response.dict()])

#transforming into a table with two rows: 'answer' and 'source'
answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

#create new dataframe with two rows: 'answer' and 'source'
structured_response_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['answer', 'source', 'reasoning'])
structured_response_df

C:\Users\Codes Soft\AppData\Local\Temp\ipykernel_17676\3395880931.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  df = pd.DataFrame([structured_response.dict()])


,paper_title,paper_summary,publication_year,paper_authors
answer,Not provided in the retrieved context,Not provided in the retrieved context,2006,D. M. Oppenheimer
source,None,None,"Oppenheimer, D. M. (2006)","Oppenheimer, D. M. (2006)"
reasoning,The title of the research paper is not mention...,A summary of the research paper is not include...,The publication year can be deduced from the c...,Only the author's name is provided in the retr...


In [32]:
print(structured_response)

paper_title='Not provided in the context' paper_summary="The paper discusses the effects of text complexity on judgments of intelligence and evaluations of writing, specifically examining how simplifications impact perceptions of an author's intelligence and the difficulty of understanding their work." publication_year='2006' paper_authors='D. M. Oppenheimer'
